# Merge data: GDP, nightlights, landcover, and NPP

## Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import json
import sys

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


## Load data

### GDP data

In [2]:
county_gdp = pd.read_csv("/work/PhD_2022_2025/data/county_rgdp.csv")
county_gdp = county_gdp.rename(
    columns={
        "Sgnyea": "year",
        "Prvcnm_id": "province_id",
        "CityName_id": "city_id",
        "Ctnm_id": "county_id"
    }
)
county_gdp = county_gdp.query("year >= 2001 and year <= 2020")
county_gdp

,year,AreaID,Ctnm_en,county_id,province_id,city_id,CityName,Prvcnm_en.x,Cmect01,Cmect02,...,Cmect14,Cmect15,Cmect16,Cmect17,Cmect18,Prvcnm_en.y,Pi0101,rel_cpi_2010,county_rgdp,county_rgdppc
1982,2001,20408037.0,Jiangpu County,320122.0,320000.0,320100.0,南京市,Jiangsu (province),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Jiangsu,100.900800,81.354509,3.951840e+05,13230.981377
1983,2001,20405086.0,Liuhe District,320116.0,320000.0,320100.0,南京市,Jiangsu (province),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Jiangsu,100.900800,81.354509,6.157004e+05,9027.157862
1984,2001,20405087.0,Lishui District,320117.0,320000.0,320100.0,南京市,Jiangsu (province),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Jiangsu,100.900800,81.354509,5.303947e+05,13126.500383
1985,2001,20405088.0,Gaochun District,320118.0,320000.0,320100.0,南京市,Jiangsu (province),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Jiangsu,100.900800,81.354509,5.303947e+05,12407.425308
1986,2001,20405097.0,Jiangyin(city),320281.0,320000.0,320200.0,无锡市,Jiangsu (province),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Jiangsu,100.900800,81.354509,4.486906e+06,38922.243153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47302,2020,NaN,Baochi District,NaN,120000.0,NaN,NaN,Tianjin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tianjin,154.111561,127.979599,2.768977e+06,NaN
47303,2020,NaN,Binhai New District,NaN,120000.0,NaN,NaN,Tianjin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tianjin,154.111561,127.979599,4.587498e+07,NaN
47304,2020,NaN,Ninghe County,NaN,120000.0,NaN,NaN,Tianjin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tianjin,154.111561,127.979599,2.443967e+06,NaN
47305,2020,NaN,Jinghai County,NaN,120000.0,NaN,NaN,Tianjin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tianjin,154.111561,127.979599,3.383209e+06,NaN


In [3]:
city_gdp = pd.read_csv("/work/PhD_2022_2025/data/city_rgdp.csv")
city_gdp = city_gdp.rename(
    columns={
        "Sgnyea": "year",
        "Prvcnm_id": "province_id",
        "Ctnm_id": "city_id"
    }
)
city_gdp = city_gdp.query("year >= 2001 and year <= 2020")
city_gdp

,year,Ctnm_en,city_id,Cttyp_en,province_id,Prvcnm_en.x,Gdpct01,Gdpct02,Gdpct03,Gdpct04,GDP_PerCapita,GDPYoY,GDP_PrimInd,GDP_SecInd,GDP_TerInd,Prvcnm_en.y,Pi0101,rel_cpi_2010,city_rgdp,city_rgdppc
327,2001,Hefei(city),340100.0,City (Sub-provincial level),340000.0,Anhui (province),363.4400,10.5000,49.3000,40.2000,8257.0000,12.5000,380900.0,1791600.0,1461900.0,Anhui,101.203500,81.073141,448.286565,10184.630653
328,2001,Huaibei(city),340600.0,City (Sub-provincial level),340000.0,Anhui (province),107.3800,16.3000,48.5000,35.2000,5436.0000,8.1000,175400.0,520800.0,377600.0,Anhui,101.203500,81.073141,132.448303,6705.056586
329,2001,Bozhou(city),341600.0,City (Sub-provincial level),340000.0,Anhui (province),164.0200,41.2000,24.2000,34.6000,3109.0000,2.5000,675600.0,397700.0,566900.0,Anhui,101.203500,81.073141,202.311144,3834.808853
330,2001,Suzhou,341300.0,City (Sub-provincial level),340000.0,Anhui (province),193.2200,46.3000,21.6000,32.1000,3330.0000,8.1000,894500.0,418200.0,619500.0,Anhui,101.203500,81.073141,238.328005,4107.402213
331,2001,Bengbu(city),340300.0,City (Sub-provincial level),340000.0,Anhui (province),171.6500,23.6000,39.7000,36.7000,5067.0000,8.0000,405100.0,681800.0,629600.0,Anhui,101.203500,81.073141,211.722399,6249.912016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6977,2020,Qitaihe(city),230900.0,City (Sub-provincial level),230000.0,Heilongjiang (province),206.4351,17.4467,38.3536,44.1998,29680.8987,-2.4537,360161.0,791752.0,912438.0,Heilongjiang,155.512726,126.283559,163.469498,23503.375193
6978,2020,Mudanjiang(city),231000.0,City (Sub-provincial level),230000.0,Heilongjiang (province),831.7356,23.7271,21.4088,54.8641,35873.1673,0.3798,1973466.0,1780650.0,4563240.0,Heilongjiang,155.512726,126.283559,658.625403,28406.838989
6979,2020,Heihe(city),231100.0,City (Sub-provincial level),230000.0,Heilongjiang (province),614.3528,44.8768,12.7195,42.4036,47085.6817,3.4752,2757020.0,781428.0,2605079.5,Heilongjiang,155.512726,126.283559,486.486764,37285.678389
6980,2020,Suihua(city),231200.0,City (Sub-provincial level),230000.0,Heilongjiang (province),1150.1633,48.7139,11.3161,39.9700,30256.8808,1.9538,5602893.0,1301541.0,4597199.0,Heilongjiang,155.512726,126.283559,910.778338,23959.477400


### VIIRS-like NTL

In [4]:
county_ntl = pd.read_csv("/work/PhD_2022_2025/data/raw_data/remote_sensing/NTL_VIIRS_like/sol_by_year_by_class_by_county.csv")
county_ntl

,province,province_id,city,city_id,county,county_id,year,urban_sol,agriculture_sol,total_sol
0,内蒙古自治区,150000,锡林郭勒盟,152500,多伦县,152531,2001,276.268933,138.464377,419.385415
1,内蒙古自治区,150000,兴安盟,152200,突泉县,152224,2001,307.790284,32.534437,340.324721
2,内蒙古自治区,150000,阿拉善盟,152900,阿拉善右旗,152922,2001,16.386511,0.000000,54.006109
3,内蒙古自治区,150000,阿拉善盟,152900,阿拉善左旗,152921,2001,266.901539,296.776790,953.765278
4,内蒙古自治区,150000,阿拉善盟,152900,额济纳旗,152923,2001,318.202330,147.921867,582.211361
...,...,...,...,...,...,...,...,...,...,...
57535,香港特别行政区,810000,香港特别行政区,810000,湾仔区,810002,2020,372.911547,204.440006,1554.756150
57536,香港特别行政区,810000,香港特别行政区,810000,西贡区,810015,2020,1572.491359,210.204998,3662.051608
57537,香港特别行政区,810000,香港特别行政区,810000,油尖旺区,810005,2020,2663.580197,0.000000,2966.562961
57538,香港特别行政区,810000,香港特别行政区,810000,元朗区,810012,2020,8857.527886,1077.126799,12666.035292


In [5]:
city_ntl = pd.read_csv("/work/PhD_2022_2025/data/raw_data/remote_sensing/NTL_VIIRS_like/sol_by_year_by_class_by_city.csv")
city_ntl

,province,province_id,city,city_id,year,urban_sol,agriculture_sol,total_sol
0,黑龙江省,230000,大兴安岭地区,232700,2001,1722.572016,410.510898,2568.673074
1,台湾省,710000,台湾省,710000,2001,244729.353051,73468.688838,360433.687369
2,安徽省,340000,安庆市,340800,2001,1531.836327,17.466398,1605.739314
3,安徽省,340000,蚌埠市,340300,2001,971.260212,45.440000,1025.059697
4,安徽省,340000,亳州市,341600,2001,1245.402142,23.540433,1274.135149
...,...,...,...,...,...,...,...,...
7415,新疆维吾尔自治区,650000,图木舒克市,659003,2020,410.849998,4959.279528,5616.826487
7416,新疆维吾尔自治区,650000,五家渠市,659004,2020,752.068157,3960.125103,5145.665962
7417,新疆维吾尔自治区,650000,新星市,659011,2020,45.045333,672.240316,1235.263314
7418,澳门特别行政区,820000,澳门特别行政区,820000,2020,3625.589212,119.415003,4602.315313


### MODIS landcover: area of class

In [6]:
county_land = pd.read_csv("/work/PhD_2022_2025/data/raw_data/remote_sensing/land_cover/class_area_by_county_by_year.csv")
county_land = county_land.fillna(0)
county_land

,province,province_id,city,city_id,county,county_id,year,1,2,3,...,8,9,10,11,12,13,14,15,16,17
0,内蒙古自治区,150000,锡林郭勒盟,152500,多伦县,152531,2001,0.000000,0.000000,0.0,...,0.000000,0.000000,3867.191672,0.000000,4.148367,4.812157,0.000000,0.0,5.743598,0.000000
1,内蒙古自治区,150000,兴安盟,152200,突泉县,152224,2001,0.000000,0.000000,0.0,...,0.000000,0.000000,4733.613539,0.175061,5.423350,50.973058,0.000000,0.0,3.948507,6.484387
2,内蒙古自治区,150000,阿拉善盟,152900,阿拉善右旗,152922,2001,0.000000,0.000000,0.0,...,0.000000,0.000000,935.101436,0.000000,0.000000,4.251747,0.000000,0.0,70571.260909,0.000000
3,内蒙古自治区,150000,阿拉善盟,152900,阿拉善左旗,152921,2001,4.856993,0.000000,0.0,...,0.000000,36.311981,4890.842461,83.288071,47.443917,21.647589,0.000000,0.0,74513.519373,0.000000
4,内蒙古自治区,150000,阿拉善盟,152900,额济纳旗,152923,2001,0.000000,0.000000,0.0,...,0.000000,0.000000,637.423056,0.000000,0.000000,29.913554,0.000000,0.0,87431.971611,0.943487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57535,香港特别行政区,810000,香港特别行政区,810000,湾仔区,810002,2020,0.000000,2.503959,0.0,...,1.541341,1.832968,0.829768,0.000000,0.000000,3.322763,0.000000,0.0,0.757489,0.632877
57536,香港特别行政区,810000,香港特别行政区,810000,西贡区,810015,2020,3.529406,54.321854,0.0,...,17.526389,8.717703,3.827837,23.116727,0.230229,16.168763,0.000000,0.0,0.000000,26.028755
57537,香港特别行政区,810000,香港特别行政区,810000,油尖旺区,810005,2020,0.000000,0.000000,0.0,...,0.000000,0.000000,0.230178,0.000000,0.000000,5.567607,0.000000,0.0,2.273897,4.518223
57538,香港特别行政区,810000,香港特别行政区,810000,元朗区,810012,2020,0.000000,20.220930,0.0,...,3.264068,38.432680,14.842414,13.787846,2.717190,49.093376,2.314522,0.0,0.000000,6.664164


In [7]:
city_land = pd.read_csv("/work/PhD_2022_2025/data/raw_data/remote_sensing/land_cover/class_area_by_city_by_year.csv")
city_land = city_land.fillna(0)
city_land 

,province,province_id,city,city_id,year,1,2,3,4,5,...,8,9,10,11,12,13,14,15,16,17
0,黑龙江省,230000,大兴安岭地区,232700,2001,0.004668,0.000000,2587.373455,5386.940050,7475.837035,...,45054.263893,3607.842692,1146.545410,18.976473,554.763405,215.819802,10.483317,0.000000,0.000000,0.471849
1,台湾省,710000,台湾省,710000,2001,1237.392795,19215.986863,0.437561,20.429541,278.616039,...,1778.662029,4287.996430,538.245053,358.572432,3412.940913,2990.034746,1762.894887,1.764360,254.857677,2188.045820
2,安徽省,340000,安庆市,340800,2001,21.654009,0.000000,0.000000,302.712838,731.852724,...,3969.973953,2356.779963,36.889047,184.413435,2712.082266,183.421082,1964.213123,0.000000,4.247096,1020.225009
3,安徽省,340000,蚌埠市,340300,2001,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,168.691038,85.070672,47.889532,5205.623491,286.281569,1.529988,0.000000,0.418387,172.591680
4,安徽省,340000,亳州市,341600,2001,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,57.111660,0.000000,0.000000,8230.748120,216.064757,22.066005,0.000000,0.000000,0.115678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,新疆维吾尔自治区,650000,图木舒克市,659003,2020,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,902.492137,0.766852,437.713525,23.372424,0.000000,0.000000,452.697078,93.508245
7416,新疆维吾尔自治区,650000,五家渠市,659004,2020,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.896535,533.424954,1.742594,170.864994,13.821016,0.000000,0.000000,16.156511,6.812034
7417,新疆维吾尔自治区,650000,新星市,659011,2020,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,693.962563,0.000000,71.685659,5.751381,0.000000,20.094297,1714.536119,0.000000
7418,澳门特别行政区,820000,澳门特别行政区,820000,2020,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,6.177281,1.830088,2.563351,0.000000,12.208917,0.000000,0.000000,2.429954,27.513582


### NPP (net primary productivity)

In [8]:
county_npp = pd.read_csv("/work/PhD_2022_2025/data/raw_data/remote_sensing/NPP/anpp_by_year_by_class_by_county.csv")
county_npp

,province,province_id,city,city_id,year,county,county_id,urban_anpp,agriculture_anpp,overall_anpp
0,内蒙古自治区,150000,锡林郭勒盟,152500,2001,多伦县,152531,0.000000,243.164088,243.167809
1,内蒙古自治区,150000,兴安盟,152200,2001,突泉县,152224,139.100000,170.622539,170.614607
2,内蒙古自治区,150000,阿拉善盟,152900,2001,阿拉善右旗,152922,0.000000,77.026909,73.867637
3,内蒙古自治区,150000,阿拉善盟,152900,2001,阿拉善左旗,152921,0.000000,90.834244,79.887617
4,内蒙古自治区,150000,阿拉善盟,152900,2001,额济纳旗,152923,0.000000,47.479602,47.936949
...,...,...,...,...,...,...,...,...,...,...
57535,香港特别行政区,810000,香港特别行政区,810000,2020,湾仔区,810002,0.000000,516.750000,932.585524
57536,香港特别行政区,810000,香港特别行政区,810000,2020,西贡区,810015,465.720000,629.323100,983.464028
57537,香港特别行政区,810000,香港特别行政区,810000,2020,油尖旺区,810005,197.327073,156.940446,181.214620
57538,香港特别行政区,810000,香港特别行政区,810000,2020,元朗区,810012,515.980090,778.431159,818.715543


In [9]:
city_npp = pd.read_csv("/work/PhD_2022_2025/data/raw_data/remote_sensing/NPP/anpp_by_year_by_class_by_city.csv")
city_npp

,province,province_id,city,city_id,year,urban_anpp,agriculture_anpp,overall_anpp
0,黑龙江省,230000,大兴安岭地区,232700,2001,196.739092,350.792143,445.582477
1,台湾省,710000,台湾省,710000,2001,534.336328,779.874391,1157.081436
2,安徽省,340000,安庆市,340800,2001,418.442287,518.011525,524.297812
3,安徽省,340000,蚌埠市,340300,2001,304.182954,361.283175,358.764103
4,安徽省,340000,亳州市,341600,2001,267.492857,338.905689,338.414874
...,...,...,...,...,...,...,...,...
7415,新疆维吾尔自治区,650000,图木舒克市,659003,2020,0.000000,62.408506,61.308951
7416,新疆维吾尔自治区,650000,五家渠市,659004,2020,0.000000,114.660579,113.863737
7417,新疆维吾尔自治区,650000,新星市,659011,2020,0.000000,163.903328,155.692760
7418,澳门特别行政区,820000,澳门特别行政区,820000,2020,345.072292,660.356735,584.859624


### Population

In [10]:
city_pop = pd.read_csv(
    "/work/PhD_2022_2025/data/raw_data/remote_sensing/population/population_by_year_by_class_by_city.csv"
)
city_pop


,province,province_id,city,city_id,year,agriculture_population,urban_population,total_population
0,黑龙江省,230000,大兴安岭地区,232700,2001,5.072447e+04,1.464841e+05,5.324005e+05
1,台湾省,710000,台湾省,710000,2001,3.995813e+06,1.434193e+07,2.230316e+07
2,安徽省,340000,安庆市,340800,2001,2.909328e+06,5.496513e+05,5.236882e+06
3,安徽省,340000,蚌埠市,340300,2001,2.371705e+06,8.565965e+05,3.373788e+06
4,安徽省,340000,亳州市,341600,2001,4.586640e+06,5.117309e+05,5.140829e+06
...,...,...,...,...,...,...,...,...
7415,新疆维吾尔自治区,650000,图木舒克市,659003,2020,1.014564e+05,3.967900e+04,1.418041e+05
7416,新疆维吾尔自治区,650000,五家渠市,659004,2020,7.413482e+04,2.167895e+04,9.983193e+04
7417,新疆维吾尔自治区,650000,新星市,659011,2020,1.129558e+04,4.596604e+03,1.704556e+04
7418,澳门特别行政区,820000,澳门特别行政区,820000,2020,0.000000e+00,4.130053e+05,5.765733e+05


In [11]:
county_pop = pd.read_csv(
    "/work/PhD_2022_2025/data/raw_data/remote_sensing/population/population_by_year_by_class_by_county.csv"
)
county_pop

,province,province_id,city,city_id,year,county,county_id,agriculture_population,urban_population,total_population
0,内蒙古自治区,150000,锡林郭勒盟,152500,2001,多伦县,152531,110874.925490,752.000000,111729.925490
1,内蒙古自治区,150000,兴安盟,152200,2001,突泉县,152224,283267.866667,43276.823529,326602.533333
2,内蒙古自治区,150000,阿拉善盟,152900,2001,阿拉善右旗,152922,343.792157,7.000000,21379.368627
3,内蒙古自治区,150000,阿拉善盟,152900,2001,阿拉善左旗,152921,28093.976471,16456.164706,152095.337255
4,内蒙古自治区,150000,阿拉善盟,152900,2001,额济纳旗,152923,995.000000,986.000000,18063.380392
...,...,...,...,...,...,...,...,...,...,...
57535,香港特别行政区,810000,香港特别行政区,810000,2020,湾仔区,810002,0.000000,167828.360784,314142.937255
57536,香港特别行政区,810000,香港特别行政区,810000,2020,西贡区,810015,7123.380392,112913.121569,198274.368627
57537,香港特别行政区,810000,香港特别行政区,810000,2020,油尖旺区,810005,0.000000,430535.890196,463232.058824
57538,香港特别行政区,810000,香港特别行政区,810000,2020,元朗区,810012,38622.156863,702043.407843,877519.031373


### Administrative boundaries

In [12]:
# Shapefile: adm3
gdf_county = gpd.read_file("/work/PhD_2022_2025/data/raw_data/map/县.shp", encoding='utf-8')
gdf_county = gdf_county.rename(
    columns={
        "省代码": "province_id",
        "市代码": "city_id",
        "县代码": "county_id",
    }
)
gdf_county

,省,province_id,市,city_id,县,county_id,说明,年度,县类型,市类型,省类型,geometry
0,安徽省,340000,安庆市,340800,大观区,340803,None,0,市辖区,地级市,省,"POLYGON ((117.05275 30.53093, 117.05199 30.525..."
1,安徽省,340000,安庆市,340800,怀宁县,340822,None,0,县,地级市,省,"POLYGON ((116.98927 30.73981, 116.98422 30.735..."
2,安徽省,340000,安庆市,340800,潜山市,340882,撤销潜山县，设立县级潜山市，以原潜山县的行政区域为潜山市行政区域，潜山市人民政府驻梅城镇潜阳...,2018,县级市,地级市,省,"POLYGON ((116.70456 31.04546, 116.70640 31.039..."
3,安徽省,340000,安庆市,340800,宿松县,340826,None,0,县,地级市,省,"POLYGON ((116.46186 30.12618, 116.47118 30.122..."
4,安徽省,340000,安庆市,340800,太湖县,340825,None,0,县,地级市,省,"POLYGON ((115.84645 30.75920, 115.84812 30.756..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2872,浙江省,330000,杭州市,330100,桐庐县,330122,None,0,县,地级市,省,"POLYGON ((119.88599 29.76591, 119.88570 29.761..."
2873,浙江省,330000,杭州市,330100,西湖区,330106,None,0,市辖区,地级市,省,"POLYGON ((120.08097 30.33345, 120.08366 30.332..."
2874,浙江省,330000,杭州市,330100,萧山区,330109,None,0,市辖区,地级市,省,"POLYGON ((120.64005 30.22064, 120.63780 30.217..."
2875,浙江省,330000,杭州市,330100,余杭区,330110,撤销杭州市余杭区，设立新的杭州市余杭区，以原余杭区的余杭街道、仓前街道、闲林街道、五常街道、...,2021,市辖区,地级市,省,"POLYGON ((119.76714 30.54884, 119.76583 30.546..."


In [13]:
#Shapefile: adm2
gdf_city = gpd.read_file("/work/PhD_2022_2025/data/raw_data/map/市.shp", encoding='utf-8')
gdf_city = gdf_city.rename(
    columns={
        "省代码": "province_id",
        "市代码": "city_id"
    }
)
gdf_city

,省,province_id,市,city_id,市类型,省类型,geometry
0,安徽省,340000,安庆市,340800,地级市,省,"POLYGON ((117.15075 31.07444, 117.14907 31.070..."
1,安徽省,340000,蚌埠市,340300,地级市,省,"POLYGON ((117.97350 33.33581, 117.98183 33.335..."
2,安徽省,340000,亳州市,341600,地级市,省,"POLYGON ((116.38814 33.78433, 116.38908 33.783..."
3,安徽省,340000,池州市,341700,地级市,省,"POLYGON ((118.09546 30.64345, 118.08867 30.636..."
4,安徽省,340000,滁州市,341100,地级市,省,"POLYGON ((119.17951 32.82747, 119.18347 32.812..."
...,...,...,...,...,...,...,...
366,浙江省,330000,绍兴市,330600,地级市,省,"POLYGON ((120.86294 30.27435, 120.86727 30.266..."
367,浙江省,330000,台州市,331000,地级市,省,"MULTIPOLYGON (((121.15911 28.02510, 121.16133 ..."
368,浙江省,330000,温州市,330300,地级市,省,"MULTIPOLYGON (((120.83169 27.04793, 120.83169 ..."
369,浙江省,330000,舟山市,330900,地级市,省,"MULTIPOLYGON (((122.16567 29.60948, 122.16513 ..."


## Merge data

### Non-spatial panel

In [14]:
#Merge county-level satellite datset 
county_remote = (
    county_ntl.merge(county_land, on=["county_id", "year"], suffixes=('_ntl', '_land'))
    .merge(county_npp, on=["county_id", "year"], suffixes=('_land', '_npp'))
    .merge(county_pop, on=["county_id", "year"], suffixes=('_npp', '_pop'))
)
county_remote


,province_ntl,province_id_ntl,city_ntl,city_id_ntl,county_ntl,county_id,year,urban_sol,agriculture_sol,total_sol,...,agriculture_anpp,overall_anpp,province_pop,province_id_pop,city_pop,city_id_pop,county_pop,agriculture_population,urban_population,total_population
0,内蒙古自治区,150000,锡林郭勒盟,152500,多伦县,152531,2001,276.268933,138.464377,419.385415,...,243.164088,243.167809,内蒙古自治区,150000,锡林郭勒盟,152500,多伦县,110874.925490,752.000000,111729.925490
1,内蒙古自治区,150000,兴安盟,152200,突泉县,152224,2001,307.790284,32.534437,340.324721,...,170.622539,170.614607,内蒙古自治区,150000,兴安盟,152200,突泉县,283267.866667,43276.823529,326602.533333
2,内蒙古自治区,150000,阿拉善盟,152900,阿拉善右旗,152922,2001,16.386511,0.000000,54.006109,...,77.026909,73.867637,内蒙古自治区,150000,阿拉善盟,152900,阿拉善右旗,343.792157,7.000000,21379.368627
3,内蒙古自治区,150000,阿拉善盟,152900,阿拉善左旗,152921,2001,266.901539,296.776790,953.765278,...,90.834244,79.887617,内蒙古自治区,150000,阿拉善盟,152900,阿拉善左旗,28093.976471,16456.164706,152095.337255
4,内蒙古自治区,150000,阿拉善盟,152900,额济纳旗,152923,2001,318.202330,147.921867,582.211361,...,47.479602,47.936949,内蒙古自治区,150000,阿拉善盟,152900,额济纳旗,995.000000,986.000000,18063.380392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57535,香港特别行政区,810000,香港特别行政区,810000,湾仔区,810002,2020,372.911547,204.440006,1554.756150,...,516.750000,932.585524,香港特别行政区,810000,香港特别行政区,810000,湾仔区,0.000000,167828.360784,314142.937255
57536,香港特别行政区,810000,香港特别行政区,810000,西贡区,810015,2020,1572.491359,210.204998,3662.051608,...,629.323100,983.464028,香港特别行政区,810000,香港特别行政区,810000,西贡区,7123.380392,112913.121569,198274.368627
57537,香港特别行政区,810000,香港特别行政区,810000,油尖旺区,810005,2020,2663.580197,0.000000,2966.562961,...,156.940446,181.214620,香港特别行政区,810000,香港特别行政区,810000,油尖旺区,0.000000,430535.890196,463232.058824
57538,香港特别行政区,810000,香港特别行政区,810000,元朗区,810012,2020,8857.527886,1077.126799,12666.035292,...,778.431159,818.715543,香港特别行政区,810000,香港特别行政区,810000,元朗区,38622.156863,702043.407843,877519.031373


In [15]:
#Merge city-level satellite datset 
city_remote = (
    city_ntl.merge(city_land, on=["city_id", "year"], suffixes=('_ntl', '_land'))
    .merge(city_npp, on=["city_id", "year"], suffixes=('_land', '_npp'))
    .merge(city_pop, on=["city_id", "year"], suffixes=('_npp', '_pop'))
)
city_remote

,province_ntl,province_id_ntl,city_ntl,city_id,year,urban_sol,agriculture_sol,total_sol,province_land,province_id_land,...,city_npp,urban_anpp,agriculture_anpp,overall_anpp,province_pop,province_id_pop,city_pop,agriculture_population,urban_population,total_population
0,黑龙江省,230000,大兴安岭地区,232700,2001,1722.572016,410.510898,2568.673074,黑龙江省,230000,...,大兴安岭地区,196.739092,350.792143,445.582477,黑龙江省,230000,大兴安岭地区,5.072447e+04,1.464841e+05,5.324005e+05
1,台湾省,710000,台湾省,710000,2001,244729.353051,73468.688838,360433.687369,台湾省,710000,...,台湾省,534.336328,779.874391,1157.081436,台湾省,710000,台湾省,3.995813e+06,1.434193e+07,2.230316e+07
2,安徽省,340000,安庆市,340800,2001,1531.836327,17.466398,1605.739314,安徽省,340000,...,安庆市,418.442287,518.011525,524.297812,安徽省,340000,安庆市,2.909328e+06,5.496513e+05,5.236882e+06
3,安徽省,340000,蚌埠市,340300,2001,971.260212,45.440000,1025.059697,安徽省,340000,...,蚌埠市,304.182954,361.283175,358.764103,安徽省,340000,蚌埠市,2.371705e+06,8.565965e+05,3.373788e+06
4,安徽省,340000,亳州市,341600,2001,1245.402142,23.540433,1274.135149,安徽省,340000,...,亳州市,267.492857,338.905689,338.414874,安徽省,340000,亳州市,4.586640e+06,5.117309e+05,5.140829e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,新疆维吾尔自治区,650000,图木舒克市,659003,2020,410.849998,4959.279528,5616.826487,新疆维吾尔自治区,650000,...,图木舒克市,0.000000,62.408506,61.308951,新疆维吾尔自治区,650000,图木舒克市,1.014564e+05,3.967900e+04,1.418041e+05
7416,新疆维吾尔自治区,650000,五家渠市,659004,2020,752.068157,3960.125103,5145.665962,新疆维吾尔自治区,650000,...,五家渠市,0.000000,114.660579,113.863737,新疆维吾尔自治区,650000,五家渠市,7.413482e+04,2.167895e+04,9.983193e+04
7417,新疆维吾尔自治区,650000,新星市,659011,2020,45.045333,672.240316,1235.263314,新疆维吾尔自治区,650000,...,新星市,0.000000,163.903328,155.692760,新疆维吾尔自治区,650000,新星市,1.129558e+04,4.596604e+03,1.704556e+04
7418,澳门特别行政区,820000,澳门特别行政区,820000,2020,3625.589212,119.415003,4602.315313,澳门特别行政区,820000,...,澳门特别行政区,345.072292,660.356735,584.859624,澳门特别行政区,820000,澳门特别行政区,0.000000e+00,4.130053e+05,5.765733e+05


In [16]:
#merge satellite data with gdp (city)
city_longpanel = city_remote.merge(city_gdp, on=["city_id", "year"], how="left")
city_longpanel

,province_ntl,province_id_ntl,city_ntl,city_id,year,urban_sol,agriculture_sol,total_sol,province_land,province_id_land,...,GDP_PerCapita,GDPYoY,GDP_PrimInd,GDP_SecInd,GDP_TerInd,Prvcnm_en.y,Pi0101,rel_cpi_2010,city_rgdp,city_rgdppc
0,黑龙江省,230000,大兴安岭地区,232700,2001,1722.572016,410.510898,2568.673074,黑龙江省,230000,...,8521.0,NaN,61000.0,207000.0,186000.0,Heilongjiang,99.0864,80.462761,56.423617,10589.992095
1,台湾省,710000,台湾省,710000,2001,244729.353051,73468.688838,360433.687369,台湾省,710000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,安徽省,340000,安庆市,340800,2001,1531.836327,17.466398,1605.739314,安徽省,340000,...,4387.0,5.3,620700.0,1153200.0,867500.0,Anhui,101.2035,81.073141,325.804571,5411.163216
3,安徽省,340000,蚌埠市,340300,2001,971.260212,45.440000,1025.059697,安徽省,340000,...,5067.0,8.0,405100.0,681800.0,629600.0,Anhui,101.2035,81.073141,211.722399,6249.912016
4,安徽省,340000,亳州市,341600,2001,1245.402142,23.540433,1274.135149,安徽省,340000,...,3109.0,2.5,675600.0,397700.0,566900.0,Anhui,101.2035,81.073141,202.311144,3834.808853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,新疆维吾尔自治区,650000,图木舒克市,659003,2020,410.849998,4959.279528,5616.826487,新疆维吾尔自治区,650000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7416,新疆维吾尔自治区,650000,五家渠市,659004,2020,752.068157,3960.125103,5145.665962,新疆维吾尔自治区,650000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7417,新疆维吾尔自治区,650000,新星市,659011,2020,45.045333,672.240316,1235.263314,新疆维吾尔自治区,650000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7418,澳门特别行政区,820000,澳门特别行政区,820000,2020,3625.589212,119.415003,4602.315313,澳门特别行政区,820000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#merge satellite data with gdp (county)
county_longpanel = county_remote.merge(county_gdp, on=["county_id", "year"], how="left")
county_longpanel

,province_ntl,province_id_ntl,city_ntl,city_id_ntl,county_ntl,county_id,year,urban_sol,agriculture_sol,total_sol,...,Cmect14,Cmect15,Cmect16,Cmect17,Cmect18,Prvcnm_en.y,Pi0101,rel_cpi_2010,county_rgdp,county_rgdppc
0,内蒙古自治区,150000,锡林郭勒盟,152500,多伦县,152531,2001,276.268933,138.464377,419.385415,...,NaN,NaN,NaN,NaN,NaN,Inner Mongolia,101.9078,80.196627,56255.483500,5520.182322
1,内蒙古自治区,150000,兴安盟,152200,突泉县,152224,2001,307.790284,32.534437,340.324721,...,NaN,NaN,NaN,NaN,NaN,Inner Mongolia,101.9078,80.196627,111370.021253,3690.928320
2,内蒙古自治区,150000,阿拉善盟,152900,阿拉善右旗,152922,2001,16.386511,0.000000,54.006109,...,NaN,NaN,NaN,NaN,NaN,Inner Mongolia,101.9078,80.196627,26946.270607,13001.793782
3,内蒙古自治区,150000,阿拉善盟,152900,阿拉善左旗,152921,2001,266.901539,296.776790,953.765278,...,NaN,NaN,NaN,NaN,NaN,Inner Mongolia,101.9078,80.196627,116420.108988,14632.785080
4,内蒙古自治区,150000,阿拉善盟,152900,额济纳旗,152923,2001,318.202330,147.921867,582.211361,...,NaN,NaN,NaN,NaN,NaN,Inner Mongolia,101.9078,80.196627,20948.512087,12783.580114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57535,香港特别行政区,810000,香港特别行政区,810000,湾仔区,810002,2020,372.911547,204.440006,1554.756150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57536,香港特别行政区,810000,香港特别行政区,810000,西贡区,810015,2020,1572.491359,210.204998,3662.051608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57537,香港特别行政区,810000,香港特别行政区,810000,油尖旺区,810005,2020,2663.580197,0.000000,2966.562961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57538,香港特别行政区,810000,香港特别行政区,810000,元朗区,810012,2020,8857.527886,1077.126799,12666.035292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#Save datasets
#city_longpanel.to_csv('/work/PhD_2022_2025/data/city_longpanel.csv')
#county_longpanel.to_csv('/work/PhD_2022_2025/data/county_longpanel.csv')

### Long to wide

In [21]:
city_wide = city_longpanel.pivot(
    index="city_id",
    columns="year",
    values=[
        "city_rgdp",
        "city_rgdppc"
    ],
)
city_wide.columns = [f"{var}_{year}" for var, year in city_wide.columns]
city_wide.reset_index(inplace=True)
city_wide

,city_id,city_rgdp_2001,city_rgdp_2002,city_rgdp_2003,city_rgdp_2004,city_rgdp_2005,city_rgdp_2006,city_rgdp_2007,city_rgdp_2008,city_rgdp_2009,...,city_rgdppc_2011,city_rgdppc_2012,city_rgdppc_2013,city_rgdppc_2014,city_rgdppc_2015,city_rgdppc_2016,city_rgdppc_2017,city_rgdppc_2018,city_rgdppc_2019,city_rgdppc_2020
0,110000,4259.773750,5083.990941,5903.414142,6941.635172,7824.333589,9097.604486,11043.187435,11910.132527,13210.534501,...,81757.243513,85331.809659,89656.927728,93860.654506,98325.250736,105301.997703,114183.075194,123960.065372,129988.189134,128336.023073
1,120000,2292.714441,2579.807117,3060.879450,3647.462890,4514.073150,5089.993637,5747.707574,6978.414225,7888.615627,...,82380.559935,87800.244324,91504.943282,94445.106287,95349.142106,97754.009291,99234.045524,98781.312687,NaN,79398.592335
2,130100,1348.555050,1489.405185,1692.462386,1924.506987,2067.953134,2306.499922,2566.616531,2905.771152,3093.539347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42343.663340,41552.764246
3,130200,1250.441499,1383.335531,1590.983917,1916.106576,2346.715596,2688.342581,3021.834575,3645.755546,3929.923013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69426.176633,73335.791885
4,130300,381.806706,421.241601,475.371727,534.233130,568.438857,628.286013,743.200264,828.159674,829.271560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41122.034353,42228.297752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,659010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,659011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,710000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,810000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
county_wide = county_longpanel.pivot(
    index="county_id",
    columns="year",
    values=[
        "county_rgdp",
        "county_rgdppc"
    ],
)
county_wide.columns = [f"{var}_{year}" for var, year in county_wide.columns]
county_wide.reset_index(inplace=True)
county_wide

,county_id,county_rgdp_2001,county_rgdp_2002,county_rgdp_2003,county_rgdp_2004,county_rgdp_2005,county_rgdp_2006,county_rgdp_2007,county_rgdp_2008,county_rgdp_2009,...,county_rgdppc_2011,county_rgdppc_2012,county_rgdppc_2013,county_rgdppc_2014,county_rgdppc_2015,county_rgdppc_2016,county_rgdppc_2017,county_rgdppc_2018,county_rgdppc_2019,county_rgdppc_2020
0,110101,NaN,NaN,NaN,NaN,NaN,6.416803e+06,7.130872e+06,7.577926e+06,1.149293e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110102,NaN,NaN,NaN,NaN,NaN,1.062305e+07,1.304934e+07,1.382988e+07,1.859137e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110105,NaN,NaN,NaN,NaN,NaN,1.572614e+07,1.797980e+07,1.921869e+07,2.437513e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110106,NaN,NaN,NaN,NaN,NaN,4.488694e+06,4.906754e+06,5.144945e+06,6.424174e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110107,NaN,NaN,NaN,NaN,NaN,2.207413e+06,2.398122e+06,2.152013e+06,2.546293e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,820005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2873,820006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2874,820007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2875,820008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99321db0-dfa2-48ce-ab9d-1e777e417e2b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>